In [2]:
using LinearAlgebra
using Optim
using Plots

# Custom factorial function for BigFloat
function factorial_big(x::BigFloat)
    if x == BigFloat(0)
        return BigFloat(1.0)
    else
        return x * factorial_big(x - BigFloat(1))
    end
end

# Set up the points with BigFloat
p1 = BigFloat[0.0, 0.0]
p2 = BigFloat[1.0, 0.0]
p3 = BigFloat[0.0, 1.0]

m1 = (p2 .+ p3) ./ BigFloat(2.0)
m2 = (p1 .+ p3) ./ BigFloat(2.0)
m3 = (p1 .+ p2) ./ BigFloat(2.0)

b = (p1 .+ p2 .+ p3) ./ BigFloat(3.0)

# vertices
T1 = hcat(p1, p2, p3)
eval_T1(i, j) = T1[1, :] .^ BigFloat(i) .* T1[2, :] .^ BigFloat(j)

# edge midpoints
T2 = hcat(m1, m2, m3)
eval_T2(i, j) = T2[1, :] .^ BigFloat(i) .* T2[2, :] .^ BigFloat(j)

# midpoint
T3 = hcat(b)
eval_T3(i, j) = T3[1, :] .^ BigFloat(i) .* T3[2, :] .^ BigFloat(j)

# edge class
T4(a) = hcat(a * p1 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p1,
             a * p3 .+ (1 .- a) * p1,
             a * p1 .+ (1 .- a) * p3,
             a * p3 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p3)

eval_T4(a, i, j) = T4(a)[1, :] .^ (0 * (i < 0) + BigFloat(i) * (i >= 0)) .* T4(a)[2, :] .^ (0 * (j < 0) + BigFloat(j) * (j >= 0))

dT4 = hcat(p1 .- p2, p2 .- p1, p3 .- p1, p1 .- p3, p3 .- p2, p2 .- p3)
eval_dT4(i, j) = dT4[1, :] .^ (0 * (i < 0) + BigFloat(i) * (i >= 0)) .* dT4[2, :] .^ (0 * (j < 0) + BigFloat(j) * (j >= 0))

# inner class, type 1
T5(a) = hcat(a * m1 .+ (1 .- a) * p1, a * m2 .+ (1 .- a) * p2, a * m3 .+ (1 .- a) * p3)
dT5 = hcat(m1 .- p1, m2 .- p2, m3 .- p3)
eval_T5(a, i, j) = T5(a)[1, :] .^ (0 * (i < 0) + BigFloat(i) * (i >= 0)) .* T5(a)[2, :] .^ (0 * (j < 0) + BigFloat(j) * (j >= 0))

# Integral function using custom factorial_big
function integral(i, j)
    factorial_big(BigFloat(j)) * factorial_big(BigFloat(i)) / factorial_big(BigFloat(i + j + 2))
end

# System setup
function Aij(i, j, a)
    vcat(sum(eval_T1(i, j)),
         sum(eval_T4(a[1], i, j)),
         sum(eval_T5(a[2], i, j)),
         sum(eval_T5(a[3], i, j)))
end

A(a,order) = hcat([Aij(i, j, a) for i in 0:order for j in 0:order if i + j <= order]...)
rhs(order) = vcat([integral(i, j) for i in 0:order for j in 0:order if i + j <= order]...)

rhs (generic function with 1 method)

In [6]:
order = 6
# f(a) = A(a,order)'*((A(a,order) * A(a,order)')\(A(a,order) * rhs(order)))-rhs(order)
f(a) = norm(A(a,order)'*((A(a,order) * A(a,order)')\(A(a,order) * rhs(order)))-rhs(order))

f (generic function with 1 method)

In [25]:
# rhs(6)

In [7]:
# Define the bounds
lower_bounds = BigFloat[0.0, 0.0, 0.0]
upper_bounds = BigFloat[Inf, Inf, Inf]

# Set up the optimization problem with bounds
# result = optimize(f, lower_bounds, upper_bounds, BigFloat[0.1, 0.1, 0.2], Fminbox(NelderMead()), Optim.Options(g_tol = 1e-63, iterations = 100_000, show_trace = false))
result = optimize(f, lower_bounds, upper_bounds, BigFloat[0.1, 0.1, 0.2], Fminbox(NelderMead()), Optim.Options(g_tol = 1e-63, iterations = 100_000, show_trace = false))

 * Status: success

 * Candidate solution
    Final objective value:     9.185104e-64

 * Found with
    Algorithm:     Fminbox with Nelder-Mead

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.41e-02 ≰ 1.0e-63

 * Work counters
    Seconds run:   52  (vs limit Inf)
    Iterations:    3
    f(x) calls:    5910
    ∇f(x) calls:   1


In [8]:
a = Optim.minimizer(result)

3-element Vector{BigFloat}:
 0.3077459416259916461046162842462509600382936084322436979861117990325812663252404
 0.2372273727931857363813267950086108322143854707884946004397633209324842310231048
 0.850680251979494304050862307988284886497296342293827048689360019564192889270086

In [11]:
(A(a,order) * A(a,order)')\(A(a,order) * rhs(order))

4-element Vector{BigFloat}:
 0.003565179653602241016812012063893264122299237666447620688443634664172285231010069
 0.01478470808840264696637773783215089794492942347110195357270891240225364353337022
 0.05094232651347590707570190028215320579175092728676235295992276755133328901377686
 0.08258974432278322464139727865631840086275765477125278587288243964406721305692034